In [1]:
import yfinance as yf
from pandas_datareader import data as pdr
import pandas as pd
yf.pdr_override() # <== the override :-)

In [49]:
exclude = ['TRIMODE', 'GREATEC', 'SNS', 'YENHER', 'LHI', 'FSBM']

tickers = [
    "GENETEC (0104)"
# "    QL (7084)  MAIN  QL RESOURCES BERHAD",
# "GREENYB (0136)  MAIN  GREENYIELD BERHAD",
# "YENHER (5300)  MAIN  YENHER HOLDINGS BERHAD",
# "CAB (7174)  MAIN  CAB CAKARAN CORPORATION BERHAD",
# "LHI (6633)  MAIN  LEONG HUP INTERNATIONAL BERHAD",
# "MCEHLDG (7004)  MAIN  MCE HOLDINGS BERHAD",
# "SMISCOR (7132)  MAIN  SMIS CORPORATION BERHAD",
# "PECCA (5271)  MAIN  PECCA GROUP BERHAD",
# "BAUTO (5248)  MAIN  BERMAZ AUTO BERHAD",
# "HLIND (3301)  MAIN  HONG LEONG INDUSTRIES BERHAD",
# "DELEUM (5132)  MAIN  DELEUM BERHAD",
# "DIALOG (7277)  MAIN  DIALOG GROUP BERHAD",
# "YINSON (7293)  MAIN  YINSON HOLDINGS BERHAD",
# "ARMADA (5210)  MAIN  BUMI ARMADA BERHAD",
# "DAYANG (5141)  MAIN  DAYANG ENTERPRISE HOLDINGS BERHAD",
# "ELSOFT (0090)  MAIN  ELSOFT RESEARCH BERHAD",
# "SNS (0259)  ACE  SNS NETWORK TECHNOLOGY BERHAD",
# "PENTA (7160)  MAIN  PENTAMASTER CORPORATION BERHAD",
# "GREATEC (0208)  MAIN  GREATECH TECHNOLOGY BERHAD",
# "FSBM (9377)  MAIN  FSBM HOLDINGS BERHAD",
# "EATECH (5259)  MAIN  E.A.TECHNIQUE (M) BERHAD",
# "TASCO (5140)  MAIN  TASCO BERHAD",
# "HARBOUR (2062)  MAIN  HARBOUR-LINK GROUP BERHAD",
# "SYSCORP (5173)  MAIN  SHIN YANG SHIPPING CORPORATION BERHAD",
# "TRIMODE (0199)  ACE  TRI-MODE SYSTEM (M) BERHAD",
# "OPENSYS (0040)  MAIN  OPENSYS (M) BERHAD",
# "CENSOF (5195)  MAIN  CENSOF HOLDINGS BERHAD",
# "REXIT (0106)  ACE  REXIT BERHAD",
# "HONGSENG (0041)  MAIN  HONG SENG CONSOLIDATED BERHAD",
# "WILLOW (0008)  MAIN  WILLOWGLEN MSC BERHAD",
# "MAYBANK (1155)  MAIN  MALAYAN BANKING BERHAD",
# "SUPERMX (7106)  MAIN  SUPERMAX CORPORATION BERHAD",
# "UCHITEC (7100)  MAIN  UCHI TECHNOLOGIES BERHAD",
# "TOPGLOV (7113)  MAIN  TOP GLOVE CORPORATION BHD",
# "INARI (0166)  MAIN  INARI AMERTRON BERHAD",
# "VITROX (0097)  MAIN  VITROX CORPORATION BERHAD"
]
tickers = {ticker.strip().split(" ")[1][1:-1] + '.KL' : ticker.strip().split(" ")[0] for ticker in tickers if ticker.strip().split(" ")[0] not in exclude}
tickers

{'0104.KL': 'GENETEC'}

In [53]:
compaines = pdr.get_data_yahoo(list(tickers.keys()), start="1999-01-01", end="2023-04-01")['Adj Close']
try:
    compaines = compaines.rename(columns = lambda x: tickers[x])
except TypeError:
    compaines = compaines.to_frame()
compaines

[*********************100%***********************]  1 of 1 completed


,Adj Close
Date,
2005-11-07,0.257982
2005-11-08,0.257982
2005-11-09,0.254757
2005-11-10,0.235408
2005-11-11,0.235408
...,...
2023-03-23,2.820000
2023-03-24,2.830000
2023-03-27,2.740000


In [60]:
nw_compaines = compaines.resample("1M").agg( lambda x: x[-1])
sample = nw_compaines.iloc[-61:, :]
sample

,Adj Close
Date,
2018-03-31,0.099379
2018-04-30,0.089665
2018-05-31,0.090412
2018-06-30,0.089665
2018-07-31,0.096390
...,...
2022-11-30,2.600000
2022-12-31,2.390000
2023-01-31,2.650000


In [63]:
sample.rename(columns={'Adj Close' : "GENETEC"}).to_clipboard(excel=True, index=False)

In [59]:
sample = sample.pct_change()
averageReturn = sample.mean()
std_return = sample.std(ddof=1)
cv = std_return / averageReturn
print(averageReturn * 100)
print(std_return * 100)
print(cv)

Adj Close    9.093099
dtype: float64
Adj Close    33.501868
dtype: float64
Adj Close    3.684318
dtype: float64


In [34]:
# nw_df = nw_compaines.iloc[-61:]
# [ticker for ticker, na in dict(nw_df.isna().any()).items() if na == True]

['TRIMODE', 'GREATEC', 'SNS', 'YENHER', 'LHI', 'FSBM']

In [5]:
import os.path

In [48]:

filename = "nw_companies.xlsx"
if not os.path.exists(filename):
    with pd.ExcelWriter(filename, engine="openpyxl", mode='w') as wf:    
        df = nw_compaines.iloc[-61:, :]
        # df.name = 'Adj Close'
        df.to_excel(wf, sheet_name=ticker)
else:
    with pd.ExcelWriter(filename, engine="openpyxl", mode='a', if_sheet_exists="replace") as wf:    
        df = nw_compaines.iloc[-61:, :]
        # df.name = 'Adj Close'
        df.to_excel(wf, sheet_name=ticker)

In [19]:
members = ['ty', 'es', 'jb', 'zk', 'extra']
comps = nw_compaines.columns
distributions = {}
interval = 7
for i, chunk in enumerate(range(0, len(comps), interval)):
    distributions[members[i]] = list(comps[chunk: interval].values)
    interval += 7
distributions


{'ty': ['WILLOW',
  'OPENSYS',
  'HONGSENG',
  'ELSOFT',
  'REXIT',
  'GREENYB',
  'TRIMODE'],
 'es': ['GREATEC', 'SNS', 'HARBOUR', 'HLIND', 'DELEUM', 'TASCO', 'DAYANG'],
 'jb': ['SYSCORP', 'CENSOF', 'ARMADA', 'BAUTO', 'EATECH', 'PECCA', 'YENHER'],
 'zk': ['LHI', 'MCEHLDG', 'QL', 'SMISCOR', 'PENTA', 'CAB', 'DIALOG'],
 'extra': ['YINSON', 'FSBM']}

In [22]:
import pyperclip

In [26]:
pyperclip.copy(", ".join(distributions['zk']))

In [2]:
market_index = pdr.get_data_yahoo("^KLSE", start="1999-01-01", end="2023-03-30")['Adj Close']
market_index

[*********************100%***********************]  1 of 1 completed


Date
1999-01-04     561.650024
1999-01-05     567.890015
1999-01-06     586.479980
1999-01-07     585.890015
1999-01-08     591.700012
                 ...     
2023-03-23    1410.979980
2023-03-24    1399.699951
2023-03-27    1396.599976
2023-03-28    1409.089966
2023-03-29    1420.349976
Name: Adj Close, Length: 5949, dtype: float64

In [3]:
market_index.resample("1M").agg( lambda x: x[-1])[-61:].to_excel("market_index.xlsx")

In [4]:
import subprocess

In [5]:
subprocess.Popen(r"market_index.xlsx", shell=True)

<Popen: returncode: None args: 'market_index.xlsx'>